#### Steady State Kalman Filter Equations

> $\dot{P}(t) = AP(t) + P(t)A^{T} + GQG^{T} - P(t)H^{T}R^{-1}HP(t)$

> $\dot{\hat{x}}(t) = A\hat{x}(t) + Bu(t) + P(t)H^{T}R^{-1}[z(t) - H\hat{x}(t)]$
>> $K(t) = P(t)H^TR^{-1}$

> $\dot{e}(t) = \dot{x}(t)-\dot{\hat{x}} = (A-K(t)H)e(t) \ + Gw \\ \dot{P}_e = (A-K(t)H)P_e(A-K(t)H)^T + GQG^T$

Follow this link for complete explanation of what's being done here: 
https://www.notion.so/Steady-State-Kalman-Gain-and-Averaging-602bd1237b5044049e2e85d17af2b2f8

>$K = PH^TR^{-1}\\ where: \ \ 0 = AP + PA^{T} + GQG^{T} - PH^{T}R^{-1}HP\\ note: \ P = P(t=\infty)$


In [ ]:
# Explore the steady state covariance
import numpy as np
import control as ct
import sympy
from sympy import Matrix, MatrixSymbol, solve

# constants 
c = 2.99792458e8;
L5 = 1176.45e6;
lam5 = c/L5;

# clock noise information
CN0dBHz = -0
CN0 = 10**(CN0dBHz/10)

h0 = 1.5e-22
h0 = h0*20    # *20 w/ vibration
h_2 = 8.5e-32

Sf = h0/2*L5**2
Sg = 2*np.pi**2*h_2*L5**2

# noise terms for SS system
Q_elem = [Sf, Sg]
Q = sympy.diag(*Q_elem)
R = Matrix([[1/CN0]])

# dynamic and measurement model information
nq, _ = Q.shape
H = Matrix([[1,0]])
A = Matrix([[0,1],[0,0]])
G = sympy.eye(nq)

# we define the covariance in symbolic form 
n, m = A.shape
P = MatrixSymbol('P', n, m)

# define algebraic Riccati equation (ARE) that dictates steady state covariance
are = sympy.Eq(A*P + P*A.T + G*Q*G.T - P*H.T*R.inv()*H*P,0)
solution = solve(are, P)

def sympy_to_control_tf(tf_matrix):
    s = sympy.symbols('s')
    tf_sys = []

    for tf in tf_matrix:
        num, den = sympy.fraction(tf)
        num_coeffs = sympy.Poly(num, s).all_coeffs()
        den_coeffs = sympy.Poly(den, s).all_coeffs()

        tf_sys.append(ct.TransferFunction(num_coeffs, den_coeffs))

    return tf_sys

if not solution:
    print("No solution found. Exiting process.")
else: 
    # process the solution 
    print("Solution found. Continuing process.")
    # the solution to the ARE represents P(t=inf) 
    P_inf = solution
    print(solution)
    K_inf = P_inf*H.T*R.inv()
    
    # next, we can determine what the transfer functions for the system are :
    s = sympy.symbols('s')
    Omega = A - K_inf*H
    Tfs = (s*sympy.eye(n) - Omega).inv()

    # get control toolbox-compatable transfer functions 
    ct_Tfs = sympy_to_control_tf(Tfs)

In [ ]:
# Calculate the observability matrix
n = A.shape[0]  # Number of states
obs_matrices = [H * A**i for i in range(n)]
observability_matrix = Matrix.hstack(*obs_matrices)

# Check the rank of the observability matrix
observability_rank = observability_matrix.rank()

# Check observability
if observability_rank == n:
    print("The system is observable.")
else:
    print("The system is not observable.")

In [ ]:
import numpy as np
import control as ct
from scipy.linalg import solve_continuous_are
import matplotlib.pyplot as plt

# constants 
c = 2.99792458e8;
L5 = 1176.45e6;
lam5 = c/L5;

# clock noise information
CN0dBHz = -0
CN0 = 10**(CN0dBHz/10)

h0 = 1.5e-22
h0 = h0*20    # *20 w/ vibration
h_2 = 8.5e-32

Sf = h0/2*L5**2
Sg = 2*np.pi**2*h_2*L5**2

# Define system matrices
A = np.array([[0, 1], [0, 0]])  # State transition matrix
B = np.array([[0], [0]])  # Control input matrix
C = np.array([[1, 0]])  # Observation matrix
Q = np.array([[Sf, 0], [0, Sg]])  # Process noise covariance
G = np.array([[1,0],[0,1]])
R = np.array([[1/CN0]])  # Measurement noise covariance

# Compute steady-state covariance matrix
P = solve_continuous_are(A.T, C.T, G@Q.T@G.T, R.T)

# Compute steady-state Kalman gain
K_ss = np.dot(P, C.T) / R

s = sympy.symbols('s')
I = sympy.eye(A.shape[0])

# contruct the steady state transfer functions 
T_ss = (s * I - (A - K_ss * C)).inv()

print("Steady-state Kalman gain:")
print(K_ss)


In [ ]:
import numpy as np
import sympy 
import control as ct
from scipy.linalg import solve_continuous_are
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

# constants 
c = 2.99792458e8;
L5 = 1176.45e6;
lam5 = c/L5;

# clock noise information
CN0dBHz = -0
CN0 = 10**(CN0dBHz/10)

h0 = 1.5e-22
h0 = h0*20    # *20 w/ vibration
h_2 = 8.5e-32

Sf = h0/2*L5**2
Sg = 2*np.pi**2*h_2*L5**2

# Define system matrices
A = np.array([[0, 1], [0, 0]])  # State transition matrix
B = np.array([[0], [0]])  # Control input matrix
C = np.array([[1, 0]])  # Observation matrix
Q = np.array([[Sf, 0], [0, Sg]])  # Process noise covariance
R = np.array([[1/CN0]])  # Measurement noise covariance

def ssResponse(T_ss):
    # Create a list to store the control transfer functions and time constants
    transfer_functions = []
    time_constants = []
    s = sympy.symbols('s')
    rows, cols = T_ss.shape
    for i in range(rows):
        for j in range(cols):
            # Extract the numerator and denominator of the transfer function
            element = T_ss[i,j]
            numerator, denominator = sympy.fraction(element)

            # Cast numerator and denominator as polynomials
            numerator_poly = sympy.Poly(numerator, s)
            denominator_poly = sympy.Poly(denominator, s)

            # get coefficients of s in format usable for control systems library
            numerator_coeffs = numerator_poly.all_coeffs()
            denominator_coeffs = denominator_poly.all_coeffs()

            # Convert coefficients to lists for control systems library
            numerator_coeffs = [float(coeff) for coeff in numerator_coeffs]
            denominator_coeffs = [float(coeff) for coeff in denominator_coeffs]

            # make transfer functions
            tf = ct.TransferFunction(numerator_coeffs, denominator_coeffs)

            print(tf)

            # Find the dominant pole
            poles = ct.pole(tf)
            dominant_pole = np.max(poles)

            # Calculate the time constant
            time_constant = 1 / np.real(dominant_pole)

            # Print the time constant
            #print("Time constant:", time_constant)

            # append for analysis
            transfer_functions.append(tf)
            time_constants.append(time_constant)

        num_plots = len(transfer_functions)

        # Create a new figure for each transfer function
        figs = []

        for g, tf in enumerate(transfer_functions):
            fig, axs = plt.subplots(2, 1)

            # Compute the magnitude, phase, and frequency
            mag, phase, omega = ct.bode_plot(tf)

            # Convert phase from radians to degrees
            phase_deg = np.degrees(phase)

            # Plot the magnitude response
            axs[0].semilogx(omega, mag)
            axs[0].set_xlabel('Frequency [rad/s]')
            axs[0].set_ylabel('Magnitude [dB]')
            axs[0].grid(True)

            # Plot the phase response
            axs[1].semilogx(omega, phase_deg)
            axs[1].set_xlabel('Frequency [rad/s]')
            axs[1].set_ylabel('Phase [degrees]')
            axs[1].grid(True)

            figs.append(fig)

            # Close the figures to prevent automatic display
            plt.close(fig)

        for fig in figs:
            fig.tight_layout()

        return figs, transfer_functions, time_constants

# simulate varying signal to noise ratios 
def varyNoise( A : np.ndarray, C : np.ndarray, Q : np.ndarray, noise_range : tuple[int,int] ): 
    # define the range of noises we will test
    lower_limit, upper_limit = noise_range
    step_size = 5
    noise_tests = np.arange(lower_limit, upper_limit, step_size)

    # get key metrics for each noise term
    tau_theta_epoch = []
    P_infs = []
    noise_term = []

    for i, sig_noise in enumerate(noise_tests):
        # define R for this epoch 
        CN0 = 10**(sig_noise/10)
        R = np.array([[1/CN0]])

        # record the measurement noise term for this test 
        noise_term.append(sig_noise)

        # Compute steady-state covariance matrix
        P = solve_continuous_are(A.T, C.T, Q.T, R.T)

        # Compute steady-state Kalman gain
        K_ss = np.dot(P, C.T) / R

        s = sympy.symbols('s')
        I = sympy.eye(A.shape[0])

        # contruct the steady state transfer functions 
        T_ss = (s * I - A + K_ss * C).inv()

        # get time constants
        bode_plts, _, taus = ssResponse(T_ss)

        # examine only the phase to LO noise time constant
        tau_theta = taus[0]
        tau_theta_epoch.append(tau_theta) # record for plotting 

        # we will record the covariane under these conditions
        P_infs.append(P[0,0]) # only examining covariance on clock phase state 

    # Plot steady-state covariance
    plt.subplot(2, 1, 1)
    plt.plot(noise_term, P_infs)
    plt.xlabel('CN0')
    plt.ylabel('Steady-State Covariance (radians^2)')
    plt.grid(True)

    # Plot time constant
    plt.subplot(2, 1, 2)
    plt.plot(noise_term, tau_theta_epoch)
    plt.xlabel('CN0')
    plt.ylabel('Time Constant (seconds)')
    plt.grid(True)

    plt.tight_layout()
    fig = plt.gcf()  # Get the current figure object

    # Close the figures to prevent automatic display
    plt.close(fig)

    return fig, bode_plts

figures, b_plts = varyNoise(A,C,Q, [-10, 40])
figures.show()

In [ ]:
# constants 
c = 2.99792458e8;
L5 = 1176.45e6;
lam5 = c/L5;

# clock noise information
CN0dBHz = -0
CN0 = 10**(CN0dBHz/10)

h0 = 1.5e-22
h0 = h0*20    # *20 w/ vibration
h_2 = 8.5e-32

Sf = h0/2*L5**2
Sg = 2*np.pi**2*h_2*L5**2

# Define system matrices
A = np.array([[0, 1], [0, 0]])  # State transition matrix
B = np.array([[0], [0]])  # Control input matrix
C = np.array([[1, 0]])  # Observation matrix
Q = np.array([[Sf, 0], [0, Sg]])  # Process noise covariance
G = np.array([[1,0],[0,1]])
R = np.array([[1/CN0]])  # Measurement noise covariance

# Compute steady-state covariance matrix
P = solve_continuous_are(A.T, C.T, G @ Q.T @ G.T, R.T)

# Compute steady-state Kalman gain
K_ss = np.dot(P, C.T) / R
print(K_ss)

s = sympy.symbols('s')
I = sympy.eye(A.shape[0])

# contruct the steady state transfer functions 
T_ss = (s * I - (A - K_ss @ C)).inv() @ G

T_ss = T_ss[0, :]

print((A - K_ss @ C))

# get time constants
bode_plt, _, taus = ssResponse(T_ss)
bode_plt[0].show()
print(taus)

In [ ]:
print(T_ss[0,:])

In [14]:
from scipy.linalg import solve_discrete_are

# constants 
c = 2.99792458e8;
L5 = 1176.45e6;
lam5 = c/L5;

# clock noise information
CN0dBHz = -0
CN0 = 10**(CN0dBHz/10)

h0 = 1.5e-22
h0 = h0*20    # *20 w/ vibration
h_2 = 8.5e-32

Sf = h0/2*L5**2
Sg = 2*np.pi**2*h_2*L5**2

# Define system matrices
A = np.array([[0, 1], [0, 0]])  # State transition matrix
B = np.array([[0], [0]])  # Control input matrix
C = np.array([[1, 0]])  # Observation matrix
Q = np.array([[Sf, 0], [0, Sg]])  # Process noise covariance
G = np.array([[1,0],[0,1]])
R = np.array([[1/CN0]])  # Measurement noise covariance

P_inf = solve_discrete_are(A.T, C.T, Q.T, R.T)
K_ss = -A @ P_inf @ C.T @ np.linalg.inv(C @ P_inf @ C.T + R)
print(K_ss)
IKHA =  (np.eye(len(A)) - K_ss*C)*A
print(IKHA)

[[0.]
 [0.]]
[[0. 0.]
 [0. 0.]]
